# SO101 数据采集教程

本教程将指导您使用主从臂采集训练数据，包括：

1. ✅ 设置摄像头
2. ✅ 配置数据集参数
3. ✅ 录制演示数据
4. ✅ 可视化和验证数据
5. ✅ 上传到 Hugging Face Hub

**前置条件**:
- ✅ 已完成 `01_so101_setup_and_calibration.ipynb`
- ✅ 主从臂已标定
- ✅ 摄像头已连接

---
## 📦 步骤 0: 导入库和配置

In [ ]:
import os
import time
from pathlib import Path

# LeRobot 相关
from lerobot.robots.so101_follower import SO101Follower, SO101FollowerConfig
from lerobot.teleoperators.so101_leader import SO101Leader, SO101LeaderConfig
from lerobot.cameras.opencv import OpenCVCameraConfig

# 数据集相关
from lerobot.datasets import LeRobotDataset
from lerobot.scripts.lerobot_record import record

print("✅ 库导入成功！")

---
## 📹 步骤 1: 设置摄像头

### 1.1 查找可用摄像头

In [ ]:
import cv2

print("🔍 搜索可用摄像头...")
print("")

available_cameras = []

# 测试摄像头索引 0-10
for i in range(10):
    cap = cv2.VideoCapture(i)
    if cap.isOpened():
        ret, frame = cap.read()
        if ret:
            h, w = frame.shape[:2]
            print(f"✅ 摄像头 {i}: {w}x{h}")
            available_cameras.append(i)
        cap.release()

print("")
print(f"找到 {len(available_cameras)} 个可用摄像头: {available_cameras}")

### 1.2 测试摄像头画面

In [ ]:
from IPython.display import display, Image
import cv2
import matplotlib.pyplot as plt

# 选择要测试的摄像头
test_camera_index = 0  # 修改为您的摄像头索引

print(f"📸 测试摄像头 {test_camera_index}...")

cap = cv2.VideoCapture(test_camera_index)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)
cap.set(cv2.CAP_PROP_FPS, 30)

if cap.isOpened():
    ret, frame = cap.read()
    if ret:
        # 转换 BGR 到 RGB
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        
        # 显示图像
        plt.figure(figsize=(10, 7))
        plt.imshow(frame_rgb)
        plt.title(f"摄像头 {test_camera_index} - {frame.shape[1]}x{frame.shape[0]}")
        plt.axis('off')
        plt.show()
        
        print("✅ 摄像头测试成功！")
    else:
        print("❌ 无法读取摄像头画面")
    
    cap.release()
else:
    print(f"❌ 无法打开摄像头 {test_camera_index}")

### 1.3 配置摄像头参数

**SmolVLA 推荐配置**:
- 分辨率: 640x480
- FPS: 30
- 至少 1 个摄像头 (推荐 2 个)

In [ ]:
# 摄像头配置
# 注意：摄像头名称需要和训练时使用的名称一致！

camera_configs = {
    "front": OpenCVCameraConfig(
        index_or_path=0,  # 修改为您的前置摄像头索引
        width=640,
        height=480,
        fps=30,
    ),
    # 可选：添加第二个摄像头（如腕部摄像头）
    # "wrist": OpenCVCameraConfig(
    #     index_or_path=1,
    #     width=640,
    #     height=480,
    #     fps=30,
    # ),
}

print("✅ 摄像头配置完成：")
for name, config in camera_configs.items():
    print(f"  - {name}: 索引 {config.index_or_path}, {config.width}x{config.height} @ {config.fps}fps")

---
## ⚙️ 步骤 2: 配置数据集参数

In [ ]:
# ========== 重要配置 ==========

# 机械臂端口和 ID（从步骤 1 获取）
FOLLOWER_PORT = "/dev/ttyACM0"
LEADER_PORT = "/dev/ttyACM1"
FOLLOWER_ID = "my_so101_follower"
LEADER_ID = "my_so101_leader"

# 数据集配置
DATASET_NAME = "my_so101_pickplace"  # 数据集名称
TASK_DESCRIPTION = "Pick up the cube and place it on the plate."  # 任务描述（英文）
NUM_EPISODES = 50  # 要录制的 episode 数量（推荐 50+）
EPISODE_TIME_S = 30  # 每个 episode 的最大时长（秒）

# Hugging Face 配置
HF_USER = "your_hf_username"  # 修改为您的 Hugging Face 用户名
REPO_ID = f"{HF_USER}/{DATASET_NAME}"

# 本地保存路径
DATA_DIR = Path("../datasets/real_robot_demos") / DATASET_NAME
DATA_DIR.mkdir(parents=True, exist_ok=True)

print("✅ 数据集配置：")
print(f"  - 数据集名称: {DATASET_NAME}")
print(f"  - 任务描述: {TASK_DESCRIPTION}")
print(f"  - Episode 数量: {NUM_EPISODES}")
print(f"  - Episode 时长: {EPISODE_TIME_S}s")
print(f"  - HF Repo: {REPO_ID}")
print(f"  - 本地路径: {DATA_DIR}")

---
## 🎬 步骤 3: 录制数据

### 3.1 准备工作

**录制前检查清单**:

- [ ] 主从臂已连接并标定
- [ ] 摄像头已连接并测试
- [ ] 工作空间整洁，光照良好
- [ ] 物体放置在合适位置
- [ ] 已登录 Hugging Face (如需上传)

### 3.2 登录 Hugging Face (可选)

In [ ]:
# 如果您想将数据集上传到 Hugging Face Hub
from huggingface_hub import notebook_login

print("🔐 登录 Hugging Face Hub...")
notebook_login()
print("✅ 登录成功！")

### 3.3 方法 1: 使用命令行录制 (推荐)

**注意**: 由于录制是交互式过程，建议在终端运行。

In [ ]:
# 生成录制命令
record_command = f"""
lerobot-record \\
  --robot.type=so101_follower \\
  --robot.port={FOLLOWER_PORT} \\
  --robot.id={FOLLOWER_ID} \\
  --robot.cameras='{{front: {{type: opencv, index_or_path: 0, width: 640, height: 480, fps: 30}}}}' \\
  --teleop.type=so101_leader \\
  --teleop.port={LEADER_PORT} \\
  --teleop.id={LEADER_ID} \\
  --dataset.repo_id={REPO_ID} \\
  --dataset.single_task="{TASK_DESCRIPTION}" \\
  --dataset.num_episodes={NUM_EPISODES} \\
  --dataset.episode_time_s={EPISODE_TIME_S} \\
  --dataset.push_to_hub=true
"""

print("📋 请在终端运行以下命令开始录制：")
print("="*60)
print(record_command)
print("="*60)
print("")
print("💡 提示：")
print("  - 按 'q' 提前结束当前 episode")
print("  - 录制完成后，数据会自动保存并上传到 Hub")
print("  - 建议每个 episode 包含完整的任务流程")

### 3.4 方法 2: 使用 Python API 录制

In [ ]:
# 这个方法适合在 Jupyter Notebook 中录制少量数据
# 对于大规模录制，推荐使用命令行

print("🎬 开始录制数据...")
print(f"📊 目标: {NUM_EPISODES} episodes")
print("")

# 创建机械臂配置
follower_config = SO101FollowerConfig(
    port=FOLLOWER_PORT,
    id=FOLLOWER_ID,
    cameras=camera_configs,
)

leader_config = SO101LeaderConfig(
    port=LEADER_PORT,
    id=LEADER_ID,
)

print("⚠️ 由于 Jupyter 限制，建议使用终端运行录制命令")
print("")
print("如果坚持在 Notebook 中录制，请参考 LeRobot 官方文档：")
print("https://huggingface.co/docs/lerobot/en/il_robots")

---
## 📊 步骤 4: 验证和可视化数据

### 4.1 加载数据集

In [ ]:
print("📂 加载数据集...")

# 从 Hub 加载（如果已上传）
try:
    dataset = LeRobotDataset(REPO_ID)
    print(f"✅ 从 Hub 加载成功: {REPO_ID}")
except Exception as e:
    print(f"⚠️ 无法从 Hub 加载: {e}")
    print("尝试从本地加载...")
    
    # 从本地加载
    try:
        dataset = LeRobotDataset(str(DATA_DIR))
        print(f"✅ 从本地加载成功: {DATA_DIR}")
    except Exception as e:
        print(f"❌ 加载失败: {e}")
        dataset = None

if dataset:
    print("")
    print("📊 数据集信息：")
    print(f"  - Episodes: {dataset.num_episodes}")
    print(f"  - Frames: {dataset.num_frames}")
    print(f"  - FPS: {dataset.fps}")
    print(f"  - Keys: {list(dataset.features.keys())}")

### 4.2 可视化第一个 Episode

In [ ]:
if dataset and dataset.num_episodes > 0:
    import matplotlib.pyplot as plt
    import numpy as np
    
    # 获取第一个 episode 的第一帧
    episode_idx = 0
    frame_idx = 0
    
    # 加载数据
    sample = dataset[frame_idx]
    
    # 显示摄像头图像
    camera_keys = [k for k in sample.keys() if 'image' in k.lower()]
    
    if camera_keys:
        num_cams = len(camera_keys)
        fig, axes = plt.subplots(1, num_cams, figsize=(6*num_cams, 5))
        
        if num_cams == 1:
            axes = [axes]
        
        for i, cam_key in enumerate(camera_keys):
            img = sample[cam_key].numpy()
            axes[i].imshow(img)
            axes[i].set_title(f"{cam_key}")
            axes[i].axis('off')
        
        plt.suptitle(f"Episode {episode_idx} - Frame {frame_idx}")
        plt.tight_layout()
        plt.show()
    
    # 显示关节状态
    if 'observation.state' in sample:
        state = sample['observation.state'].numpy()
        print("")
        print(f"关节状态: {state}")
    
    if 'action' in sample:
        action = sample['action'].numpy()
        print(f"动作: {action}")
else:
    print("⚠️ 没有可用的数据集")

### 4.3 统计信息

In [ ]:
if dataset:
    print("📊 数据集统计：")
    print("")
    
    # Episode 时长分布
    episode_lengths = []
    for ep in range(dataset.num_episodes):
        start = dataset.episode_data_index["from"][ep]
        end = dataset.episode_data_index["to"][ep]
        length = end - start
        episode_lengths.append(length)
    
    print(f"Episode 帧数统计：")
    print(f"  - 平均: {np.mean(episode_lengths):.1f} 帧")
    print(f"  - 最小: {np.min(episode_lengths)} 帧")
    print(f"  - 最大: {np.max(episode_lengths)} 帧")
    print(f"  - 总计: {sum(episode_lengths)} 帧")
    
    # 转换为时间（假设 FPS=30）
    fps = dataset.fps
    print("")
    print(f"Episode 时长统计 (@ {fps} FPS)：")
    print(f"  - 平均: {np.mean(episode_lengths)/fps:.1f} 秒")
    print(f"  - 最小: {np.min(episode_lengths)/fps:.1f} 秒")
    print(f"  - 最大: {np.max(episode_lengths)/fps:.1f} 秒")
    
    # 绘制分布图
    plt.figure(figsize=(10, 5))
    plt.hist(episode_lengths, bins=20, edgecolor='black')
    plt.xlabel('帧数')
    plt.ylabel('Episode 数量')
    plt.title('Episode 长度分布')
    plt.grid(True, alpha=0.3)
    plt.show()

---
## 🚀 步骤 5: 上传到 Hugging Face Hub

如果您在录制时没有使用 `--dataset.push_to_hub=true`，可以手动上传。

In [ ]:
if dataset:
    print("📤 上传数据集到 Hugging Face Hub...")
    print(f"目标仓库: {REPO_ID}")
    print("")
    
    try:
        # 上传数据集
        dataset.push_to_hub(
            repo_id=REPO_ID,
            private=False,  # 设置为 True 如果您想保持私有
        )
        print("✅ 上传成功！")
        print("")
        print(f"🔗 查看数据集: https://huggingface.co/datasets/{REPO_ID}")
        print(f"🔗 可视化: https://huggingface.co/spaces/lerobot/visualize_dataset?path=/{REPO_ID}/episode_0")
    except Exception as e:
        print(f"❌ 上传失败: {e}")
        print("")
        print("请确保：")
        print("  1. 已登录 Hugging Face")
        print("  2. 仓库名称正确")
        print("  3. 有写入权限")
else:
    print("⚠️ 没有可用的数据集")

---
## ✅ 数据采集完成！

### 数据质量检查清单

在进行训练前，请确认：

- [ ] 至少有 50 个高质量 episodes
- [ ] 每个 episode 包含完整的任务流程
- [ ] 摄像头图像清晰，光照良好
- [ ] 包含多种任务变化（如物体位置）
- [ ] 动作流畅，没有异常卡顿
- [ ] 数据已成功上传到 Hugging Face Hub

### 数据质量建议

**推荐做法** (参考 `lerobot/svla_so101_pickplace`):
- 50 episodes
- 5 个不同的物体位置
- 每个位置 10 个演示
- 包含足够的变化以提高泛化能力

### 下一步

数据采集完成后，请继续：

**`03_so101_smolvla_training.ipynb`** - 训练 SmolVLA 模型

---

**🎉 祝您数据采集顺利！**